In [2]:
import pandas as pd 
import numpy as np 
from collections import Counter, defaultdict
import random
from diffprivlib.mechanisms import PermuteAndFlip
from Forest import DPRF_Forest
import sys
import math

## Datasets - Adult, Mushroom, Heart (?)

In [3]:
# Missing Attribute Values: 2480 of them (denoted by "?"), all forattribute #11.

datasetmush = 'https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
row_names = ["edible", "cap-shape", "cap-surf", "cap-col", "bruise", "odor", "gill-att", "gill-space", "gill-size",
"gill-col", "stalk-shape", "stalk-root", "st-surf-ab-ring", "st-surf-bl-ring", "st-col-ab-ring", "st-col-bl-ring",
"veil-type", "veil-col", "ring-num", "ring-type", "spore-col", "popul", "habitat"]
mushdata = pd.read_csv(datasetmush, names = row_names, na_values=[" ?"])
mushdata['edible'].replace(to_replace=['e', 'p'], value=[1, 0], inplace=True)
mushdata["edible"] = mushdata["edible"].astype(int)
Dmush = mushdata.to_numpy()

mushdata.shape

(8124, 23)

In [4]:
mushdata = mushdata[["cap-shape", "cap-surf", "cap-col", "bruise", "odor", "gill-att", "gill-space", "gill-size",
"gill-col", "stalk-shape", "stalk-root", "st-surf-ab-ring", "st-surf-bl-ring", "st-col-ab-ring", "st-col-bl-ring",
"veil-type", "veil-col", "ring-num", "ring-type", "spore-col", "popul", "habitat", "edible"]]

In [5]:
mush = mushdata[mushdata["stalk-root"] != "?"]

In [6]:
np.random.seed(97)
D2mush = mush.to_numpy()
D2mush = D2mush.astype(str)
np.random.shuffle(D2mush)

In [7]:
dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
row_names = ["Age", "Workclass", "Fnlwgt", "Education", "EducationNum", "MaritalStatus",
        "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss",
        "HoursPerWeek", "Country", "Income"]
us_adult_income = pd.read_csv(dataset, names=row_names,na_values=[' ?'])
us_adult_income["Income"] = pd.Categorical(us_adult_income["Income"])
us_adult_income["Income"] = us_adult_income["Income"].cat.codes
us_adult_income["Income"] = 1 - us_adult_income["Income"]

us_adult_income.head()

,Age,Workclass,Fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [8]:
random.seed(42)
adult2 = us_adult_income.sample(frac=1, random_state=42)
adult2.shape

(32561, 15)

In [9]:
adult25 = adult2.copy()
adult25["Workclass"] = pd.Categorical(adult25["Workclass"])
adult25["Workclass"] = adult25["Workclass"].cat.codes

adult25["Education"] = pd.Categorical(adult25["Education"])
adult25["Education"] = adult25["Education"].cat.codes

adult25["MaritalStatus"] = pd.Categorical(adult25["MaritalStatus"])
adult25["MaritalStatus"] = adult25["MaritalStatus"].cat.codes

adult25["Occupation"] = pd.Categorical(adult25["Occupation"])
adult25["Occupation"] = adult25["Occupation"].cat.codes

adult25["Relationship"] = pd.Categorical(adult25["Relationship"])
adult25["Relationship"] = adult25["Relationship"].cat.codes

adult25["Race"] = pd.Categorical(adult25["Race"])
adult25["Race"] = adult25["Race"].cat.codes

adult25["Gender"] = pd.Categorical(adult25["Gender"])
adult25["Gender"] = adult25["Gender"].cat.codes

adult25["Country"] = pd.Categorical(adult25["Country"])
adult25["Country"] = adult25["Country"].cat.codes

In [10]:
adult23 = adult25[["Age", #DONE
                    "Workclass", # DONE
                    "Education", #DONE
                    "MaritalStatus", # DONE
                    "Occupation", # DONE
                    "Relationship", # DONE
                    "Race", # DONE
                    "HoursPerWeek", # DONE
                    "Gender", # DONE
                    #"CapitalGain",
                    #"CapitalLoss",
                    "Income"]] # Education, Workclass, Occupation and Gender are discrete. 
#adult23

In [11]:
adult23 = adult23.drop(adult23[(adult23["Workclass"] == -1) |
                               (adult23["Education"] == -1) |
                               (adult23["Occupation"] == -1) |
                               (adult23["MaritalStatus"] == -1) |
                                (adult23["HoursPerWeek"] == -1) 
                            ].index)
adult23.shape

(30718, 10)

In [ ]:
#adult23 = adult23[adult23["Country"] != 14]
#adult23["Country"].value_counts()

In [102]:
#adult23 = adult23[adult23["Occupation"] != 1]
#adult23 = adult23[adult23["Occupation"] != -1]
#adult23 = adult23[adult23["Occupation"] != 8]
#adult23["Occupation"].value_counts()

In [103]:
#adult23 = adult23[adult23["Education"] != 13]
#adult23 = adult23[adult23["Education"] != 3]
# adult23["Education"].value_counts()

In [12]:
adult23 = adult23[adult23["Workclass"] != 7]  
#adult23 = adult23[adult23["Workclass"] != 6]
# only 4 people in workclass 6...
adult23["Workclass"].value_counts()

3    22696
5     2541
1     2093
6     1298
4     1116
0      960
Name: Workclass, dtype: int64

In [16]:
#adult23 = adult23[adult23["Race"] != 0]  
#adult23 = adult23[adult23["Race"] != 3]
# only 4 people in workclass 6...
#adult23["Race"].value_counts()

In [17]:
#adult23 = adult23[adult23["MaritalStatus"] != 1]  
#adult23 = adult23[adult23["MaritalStatus"] != 3]
# only 4 people in workclass 6...
#adult23["MaritalStatus"].value_counts()

In [18]:
                    #"Workclass", # DONE
                    #"Education", #DONE
                    #"MaritalStatus", # DONE
                    #"Occupation", # DONE
                    #"Relationship", # DONE
                    #"Race", # DONE
                    #"Gender"
#adult23["Workclass"].value_counts()

In [13]:
np.random.seed(97)
D2 = adult23.to_numpy()
D2 = D2.astype(str)
np.random.shuffle(D2)

In [14]:
D4 = D2.astype(str)
# D4

In [15]:
# make sure both test and training data have all classes in them
# the code will not run if D88 and D99 do not print the same arrays
D22 = D2
len(D22)

30704

In [16]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [30]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [31]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [32]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [33]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,1]).astype(int))

array([0, 1, 3, 4, 5, 6])

In [34]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,1]).astype(int))

array([0, 1, 3, 4, 5, 6])

In [35]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,3]).astype(int))

array([0, 1, 2, 3, 4, 5, 6])

In [36]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,3]).astype(int))

array([0, 1, 2, 3, 4, 5, 6])

https://github.com/loginaway/DecisionTree

In [41]:
# adult23[["CapitalGain", "CapitalLoss"]].describe()

In [16]:
len(D2)

30704

In [17]:
# from sklearn.model_selection import train_test_split

low = int(len(D2) * 0.75)
high = len(D2)

train_data = D2[0:low]
test_data = D2[low:high]

# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
print(len(test_data), len(train_data))
print(len(D2))
print(len(test_data) + len(train_data))

7676 23028
30704
30704


In [18]:
feature_discrete_mushroom = {"cap-shape":True, 
                            "cap-surf":True, 
                            "cap-col":True, 
                            "bruise":True, 
                            "odor":True, 
                            "gill-att":True, 
                            "gill-space":True, 
                            "gill-size":True, "gill-col":True, 
"stalk-shape":True, "stalk-root":True, "st-surf-ab-ring":True, "st-surf-bl-ring":True, "st-col-ab-ring":True, 
"st-col-bl-ring":True,"veil-type":True, "veil-col":True, "ring-num":True, "ring-type":True, "spore-col":True, 
"popul":True, "habitat":True}

low = int(len(D2mush) * 0.75)
high = len(D2mush)

train_data_mush = D2mush[0:low]
test_data_mush = D2mush[low:high]

# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
print(len(test_data_mush), len(train_data_mush))
print(len(D2mush))
print(len(test_data_mush) + len(train_data_mush))


1411 4233
5644
5644


In [19]:

# Input:
    # Training set D 
    # Privacy budget B (epsilon)

    # Test Set T (?)  (Maybe?)

    # Number of attributes to be split used by each dec tree f 
    # Max tree depth d_m
    # banana

B = 0.25 # epsilon
# f = 3
dm = 10
num_trees = 3


feature_discrete_adult = {'Age': False, 
                          'Workclass':True, 
                          'Education': True, 
                          "MaritalStatus":True,
                          'Occupation': True, 
                          "Relationship":True,
                          "Race":True,
                          "HoursPerWeek":False,
                          "Gender": True}
                          #"CapitalGain":False,
                          #"CapitalLoss":False}
                          
# diff_priv_forest = DPRF_Forest(adult2, num_trees, adult23, train_data, B, dm, feature_discrete_adult)

diff_priv_forest2 = DPRF_Forest( mushdata, num_trees, mush, train_data_mush, B, dm, feature_discrete_mushroom )
                            #  A_ind,      A, attribute_values, root, class_values, feature_discrete, treetype, dataset_size, epsilon_per_tree, max_depth)

# RecursionError: maximum recursion depth exceeded while calling a Python object
# TRee_DPDR:
# node['>'] = self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A, self.current_depth+1)

#feature_discrete_iris =  {'sepal length': False, 'sepal width': False, 'petal length': False, 'petal width': False}
#diff_priv_forest = DPRF_Forest(irisdata, irisdata, Diris.astype('<U21'), B, f, dm, feature_discrete_iris)

#num_trees = len(our_diff_priv_forest._trees)
#av_prunings = np.mean([x._prunings for x in our_diff_priv_forest._trees])

#av_tree_size = np.mean([x._id-x._prunings+1 for x in our_diff_priv_forest._trees])
#accuracy, leafs_not_most_confident, votes_requiring_average = diff_priv_forest.evaluate_accuracy_with_voting(adult23, class_index=0) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

# feature discrete
# {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True}

NUM TREES = 3 & EPSILON PER TREE = 0.08333333333333333
Epsilon used since tree is random :  0.25
currently training! 
chosen attr:  bruise
chosen attr:  ring-type
chosen attr:  veil-col
chosen attr:  stalk-root
chosen attr:  cap-shape
chosen attr:  popul
chosen attr:  odor
chosen attr:  cap-surf
chosen attr:  gill-col
chosen attr:  gill-col
chosen attr:  stalk-shape
chosen attr:  popul
class counts: 
defaultdict(<class 'int'>, {'1': 109, '0': 0})
noisy labels: 
None
Utility and epsilon: 
[109, 0] 0.25
noisy label after: 
1
class counts: 
defaultdict(<class 'int'>, {'0': 2, '1': 0})
noisy labels: 
None
Utility and epsilon: 
[2, 0] 0.25
noisy label after: 
0
class counts: 
defaultdict(<class 'int'>, {'0': 152, '1': 0})
noisy labels: 
None
Utility and epsilon: 
[152, 0] 0.25
noisy label after: 
0
class counts: 
defaultdict(<class 'int'>, {'0': 3, '1': 0})
noisy labels: 
None
Utility and epsilon: 
[3, 0] 0.25
noisy label after: 
0
class counts: 
defaultdict(<class 'int'>, {'1': 1, '0': 0})

In [41]:
#g = D2[400:][:,-4]
#unique, counts = np.unique(g, return_counts=True)
#print(np.asarray((unique, counts)).T)

In [229]:
test_data

array([['31', '3', '11', ..., '0', '0', '1'],
       ['29', '3', '15', ..., '1831', '0', '1'],
       ['59', '5', '11', ..., '0', '0', '1'],
       ...,
       ['68', '3', '11', ..., '0', '0', '1'],
       ['25', '3', '11', ..., '0', '0', '1'],
       ['22', '3', '11', ..., '0', '0', '1']], dtype='<U21')

In [20]:
#accuracy = diff_priv_forest.evaluate_accuracy_with_voting(test_data, class_index=11) # confidence count includes all trees (before voting)
accuracy = diff_priv_forest2.evaluate_accuracy_with_voting(test_data_mush, class_index=22)

print("accuracy = {}".format(accuracy))

result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['0']]
votes: 
[1, 1, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]


In [ ]:
# something's not right with the voting system! 
#accuracy = diff_priv_forest.evaluate_accuracy_with_voting(Diris.astype('<U21'), class_index=4) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

In [461]:
# %history -g -f hybridTree2019.ipynb

In [24]:
import statistics 

l = [1,3,4,5,3,11,18,13,6,7,8]

statistics.median(l)

6

In [ ]:
def median(data):
    """Return the median (middle value) of numeric data.

    When the number of data points is odd, return the middle data point.
    When the number of data points is even, the median is interpolated by
    taking the average of the two middle values:

    >>> median([1, 3, 5])
    3
    >>> median([1, 3, 5, 7])
    4.0

    """
    
    data = sorted(data)
    n = len(data)

    if n == 0:
        raise StatisticsError("no median for empty data")
    if n % 2 == 1:
        return data[n // 2]
    else:
        i = n // 2
        return (data[i - 1] + data[i]) / 2
